<a href="https://colab.research.google.com/github/shagags/Bitcoinwick-api/blob/main/scalping_strategy_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy fastapi uvicorn requests

In [ ]:
import pandas as pd
import numpy as np

def generate_signal(df):
    """
    Example function:
    - df: DataFrame with columns ['open','high','low','close'] at 1-minute frequency.
    - Returns 'BUY', 'SELL', or 'HOLD'.
    """
    # Calculate 5 and 13 EMA for trend filter
    df['ema5'] = df['close'].ewm(span=5, adjust=False).mean()
    df['ema13'] = df['close'].ewm(span=13, adjust=False).mean()

    # Check if 5 EMA > 13 EMA (uptrend) and last 3 bars inside a tight range
    if df['ema5'].iloc[-1] > df['ema13'].iloc[-1]:
        last3 = df['close'].iloc[-3:]
        if max(last3) - min(last3) < np.mean(df['close'].diff().abs().iloc[-50:]) * 0.5:
            if df['close'].iloc[-1] > max(last3[:-1]):
                return "BUY"
    # Reverse logic for SELL
    if df['ema5'].iloc[-1] < df['ema13'].iloc[-1]:
        last3 = df['close'].iloc[-3:]
        if max(last3) - min(last3) < np.mean(df['close'].diff().abs().iloc[-50:]) * 0.5:
            if df['close'].iloc[-1] < min(last3[:-1]):
                return "SELL"
    return "HOLD"

# Test on dummy data
test_data = pd.DataFrame({
    'open': [1,1.02,1.03,1.05,1.07],
    'high':[1.02,1.03,1.05,1.07,1.08],
    'low':[1,1.02,1.01,1.04,1.05],
    'close':[1.02,1.03,1.05,1.07,1.06]
})
print(generate_signal(test_data))  # Should output BUY, SELL, or HOLD